<a href="https://colab.research.google.com/github/xoxominji/22-1-ESAA-Practice/blob/main/0328_%EB%B0%95%EB%AF%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [개념정리]

## 핸즈온 Chapter 7. 앙상블 학습과 랜덤 포레스트

**앙상블 기법(ensemble method)**: 일련의 예측기(분류/회귀 모델)를 모아 학습하는 알고리즘



### 7.1 투표 기반 분류기


- 약한 학습기(weak learner): 랜덤 추측보다 조금 더 높은 성능을 내는 분류기
- 강한 학습기(strong learner): 높은 정확도를 냄

- **직접 투표(hard voting)**: 다수결 투표
- **간접 투표(soft voting)**: 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스로 예측 (```predict_proba()```)


```
from sklearn.ensemble import VotingClassifier
VotingClassifier(voting="hard"/"soft")
```



> 앙상블 방법은 예측기가 서로다른 알고리즘일 때, 서로 독립일 때 최고의 성능

> 앙상블의 예측 > 결정 트리 하나 예측, 더 작은 분산을 만듦

> 편향: 배깅 > 페이스팅 (상관관계 줄여 분산 감소)

### 7.2 배깅과 페이스팅
- **배깅(bootstrap aggregating)**: 중복 허용 샘플링

- **페이스팅(pasting)**: 중복 미허용 샘플링

<수집 함수>
- 분류: 통계적 최빈값
- 회귀: 평균

#### 7.2.1 사이킷런의 배깅과 페이스팅


```
from sklearn.ensemble import BassigClassifier #분류
from sklearn.ensemble import BassigRegressor #회귀

BaggingClassifier(bootstrap=False, #페이스팅
                  n_jobs=#CPU 코어 수#)

```
#### 7.2.2 oob 평가

- **oob(Out-of-Bag)**: 선택되지 않은 훈련 샘플의 나머지

예측기가 학습 중에 oob 샘플을 사용하지 않으므로 이 샘플들을 검증 세트로 사용가능 -> 앙상블 모델 자체는 예측기별 oob 평가의 평균값을 이용하여 평가할 수 있음



```
BaggingClassifier(oob_score=True)
모델.oob_score_ #출력
모델.oob_decision_function_ #oob샘플에 대한 결정 함수 값
```



### 7.3 랜덤 패치와 랜덤 서브스페이스



```
BaggingClassifier(max_samples=#, bootstrap=True/False) #샘플 샘플링
BaggingClassifier(max_features=[], bootstrap_features=[]) #특성 샘플링
```

**랜덤 패치 기법(random patches method)**: 훈련 특성 & 모두 샘플링

랜덤 서브스페이스 방식: 
- ```bootstrap=False``` 그리고 ```max_samples=1.0```
- ```bootstrap_features=True``` 또는 ```max_features < 1.0```

> 편향을 늘리는 대신 분산을 낮춤

### 7.4 랜덤 포레스트

: 배깅/페이스팅을 적용한 결정 트리의 앙상블

랜덤포레스트 = 배깅(결정 트리)

```
BaggingClassifier(
  DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
  n_estimators=500,
  max_samples=1.0, 
  bootstrap=True,
  n_jobs=-1)
```

#### 7.4.1 엑스트라 (랜덤) 트리

: 최적의 임곗값을 찾는 대신 후보 특성을 사용해 분할한 다음 최상의 분할 선택

> 편향 up, 분산 down

> speed up



```
from sklearn.ensemble import ExtraTreesClassifier
ExtraTreesClassifier(n_estimators=, max_leaf_nodes=, n_jobs, ...)
```

#### 7.4.2 특성 중요도

: 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는 확인

```rf모델.feature_importances_```

### 7.5 부스팅

: 약한 학습기 여러개 연결해 강한 학습기를 만드는 앙상블 방법, 앞 모델을 보완해나가면서 일련의 예측기 학습

#### 7.5.1 에이다부스트

: 이전 모델이 제대로 학습하지 못한 샘플들에 대한 **가중치를 조절**하여 새로운 모델을 만들어 가는 방식

가중치가 적용된 훈련 세트의 전반적인 정화도에 따라 예측기마다 다른 가중치가 적용됨

연속된 학습 기법: 영렬화/분할 불가, 배깅/페이스팅만큼 확장성이 높이 않음

```
from sklearn.ensemble import AdaBoostClassifier
AdaBoostClassifier(
  DecisionTreeClassifier(max_depth=1),
  algorithm="SAMME.R" #클래스 확률 추정
)
```



#### 7.5.2 그레이디언트 부스팅
: 전 예측기가 만든 **잔여 오차(residual error)**에 대해 새로운 예측기를 학습



```
from sklearn.ensemble import GradientBoostingRegressor/Classifier
GradientBoostingRegressor/Classifier()
```
<매개변수>

- ```max_depth``` : ```n_estimators```
- ```min_sample_leaf```
-```learning_rate```: 각 트리의 기여 정도, < 0.1 이면 '축소' 규제 방법 (훈련 세트 학습 위해 많은 트리가 필요하지만 예측 성능이 좋아짐)

- ```staged_predict(X_val)``` : 최적의 트리 수 찾기
- ```warm_start```: 조기 종료
- ```subsample```: 훈련에 사용할 훈련 샘플 비율 지정 (편향 up, 분산 down), 0.25 지정시 **확률적 그레이디언트 부스팅(stochastic gradient boosting)**


### 7.6 스태킹

"앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까?"

1. 훈련 세트를 두 개의 서브셋으로 나누기
2. 첫 번째 서브셋으로 첫번째 레이어의 예측 훈련
3. 첫 번째 레이어의 예측기로 두번째 홀드 아웃 세트에 대한 예측
4. 각 샘플에 대한 여러 개의 예측값으로 새로운 훈련 세트 생성 (n차원)
5. 새 훈련 세트로 **메타 학습기** 훈련

# [필사]

In [26]:
# moons 데이터셋 생성
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=1000, noise=0.15)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
# 다수결 분류기
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [28]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.895
RandomForestClassifier 0.98
SVC 0.985
VotingClassifier 0.985


In [29]:
#배깅
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=500,
    max_samples=100, 
    n_jobs=-1)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [30]:
# oob
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=500,
    bootstrap=True, 
    oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.985

In [31]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.98

In [32]:
bag_clf.oob_decision_function_[:10]

array([[0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.97175141, 0.02824859]])



---



---



In [33]:
#랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [34]:
bag_clf = BaggingClassifier(
      DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
      n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

In [35]:
# rf: 특성 중요도
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.10735261943150355
sepal width (cm) 0.024315584300831104
petal length (cm) 0.4542284430950481
petal width (cm) 0.4141033531726171




---



---



In [36]:
#에이다부스팅
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), 
    n_estimators=200,
    algorithm="SAMME.R", 
    learning_rate=0.5)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [37]:
#그레이디언트 부스팅
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [38]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [39]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [41]:
import numpy as np
X_new = np.array([[0.8]]) # 새로운 샘플
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3)) # 모든 예측값 더하기

ValueError: ignored

In [42]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [43]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 이전 학습 내용을 지우고 새로 훈련하기 위해 훈련 세트 섞기. 
# random_state를 다른 값으로 지정.
X_train, X_val, y_train, y_val = train_test_split(X, y)

# GBRT 모델 설정 및 훈련
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

# staged_predict()에 의해 생성된 반복자를 활용하여
# 각 단계별 MSE 수집 후 최소값을 갖는 인덱스 확인
errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

# 최적의 의사결정나무 수를 이용하여 새로 학습
gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [44]:
# n_estimators 파라미터를 지정하지 않은 채로 모델 설정
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

# MSE 최솟값 보관용
min_val_error = float("inf")

# MSE의 변화가 없거나 나빠지는 횟수 측정. 5회까지 기다리기.
error_going_up = 0

# n_estimators를 1부터 120까지 실험
for n_estimators in range(1, 120):
    # GBRT의 n_estimators 설정 후 학습
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    
    # MSE 측정
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    
    # MSE가 낮아졌을 경우 최솟값 업데이트.
    # 아니면 5번 정도 기다렸다가 조기 종료할 것.
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break

In [46]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

[08:43:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [47]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)
val_error = mean_squared_error(y_val, y_pred)

[08:44:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.462332
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.429475
[2]	validation_0-rmse:0.400964
[3]	validation_0-rmse:0.376247
[4]	validation_0-rmse:0.355539
[5]	validation_0-rmse:0.337305
[6]	validation_0-rmse:0.322274
[7]	validation_0-rmse:0.305324
[8]	validation_0-rmse:0.293274
[9]	validation_0-rmse:0.279954
[10]	validation_0-rmse:0.27075
[11]	validation_0-rmse:0.260689
[12]	validation_0-rmse:0.252376
[13]	validation_0-rmse:0.244821
[14]	validation_0-rmse:0.238213
[15]	validation_0-rmse:0.232236
[16]	validation_0-rmse:0.224774
[17]	validation_0-rmse:0.219637
[18]	validation_0-rmse:0.213365
[19]	validation_0-rmse:0.209135
[20]	validation_0-rmse:0.204201
[21]	validation_0-rmse:0.200707
[22]	validation_0-rmse:0.196584
[23]	validation_0-rmse:0.193711
[24]	validation_0-rmse:0.190399
[25]	validation_0-rm